**Before running this notebook, please add a shortcut of the RecSys folder to the root directory of your drive (MyDrive)**

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


The following is for the KAN layer.

In [2]:
!git clone https://github.com/ZiyaoLi/fast-kan

!pip install '/content/fast-kan/.'

fatal: destination path 'fast-kan' already exists and is not an empty directory.
Processing ./fast-kan
  Preparing metadata (setup.py) ... done
  Created wheel for fastkan: filename=fastkan-0.0.1-py3-none-any.whl size=11560 sha256=054fcc422dae8151f15b0aae0a3b059a13931b69382aec3106b34fc9d9e5bb8e
  Stored in directory: /root/.cache/pip/wheels/90/67/af/0e9541499b9ccc4465dae8c80e40769cc637277b82bcfeb08b
Successfully built fastkan
  Attempting uninstall: fastkan
    Found existing installation: fastkan 0.0.1
    Uninstalling fastkan-0.0.1:
      Successfully uninstalled fastkan-0.0.1


In [3]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/RecSys/')
sys.path.append('/content/drive/MyDrive/RecSys/model/')

In [4]:
import math
import random as rnd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Subset,random_split,default_collate
from sklearn.model_selection import train_test_split
import time
import logging


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
from numba import jit

In [6]:
from Recommender import AEReco
from MovieLens_4Colab import Movie_1M, CombinedDataset



I already have the pre-processed data saved, so there is no need to unquote the following blocks until we get a new dataset or run cross validation.

In [7]:
dataset = Movie_1M()

In [8]:
dataset.users.values[2]

array([3, list([0, 0, 1, 0, 0, 0, 0]), 0,
       list([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])],
      dtype=object)

In [9]:
len(dataset)

1000209

In [10]:
indices = np.random.permutation(len(dataset))
train_index = indices[:int(round(0.9*len(dataset)))] #change the train_test_split to 9:1

In [11]:
dataset.users

,user_id,age,gender,zip_code,occupation
0,1,"[1, 0, 0, 0, 0, 0, 0]",1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,2,"[0, 0, 0, 0, 0, 0, 1]",0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,"[0, 0, 1, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,4,"[0, 0, 0, 0, 1, 0, 0]",0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,5,"[0, 0, 1, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
6035,6036,"[0, 0, 1, 0, 0, 0, 0]",1,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
6036,6037,"[0, 0, 0, 0, 1, 0, 0]",1,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6037,6038,"[0, 0, 0, 0, 0, 0, 1]",1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6038,6039,"[0, 0, 0, 0, 1, 0, 0]",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
#train_rating, train_mask, test_rating, test_mask, user_matrix, item_matrix = dataset.preprocessor(train_index, fourier = False)

Loading the Model ... the configuration file should be pretty straightforward

The configuration file: MovieLens1M.yaml

> Add blockquote



For fine-tuning: please change parameters from the yaml so that we don't have to restart the run time every time a hyper-parameter is changed.

In [13]:
model = AEReco(configuration_file = '/content/drive/MyDrive/RecSys/MovieLens1M.yaml')


 	 ----------- Model = Recommender with Autoencoder ------------

 	 	 Autoencoder = DAE_KAN 

 	 ----------- Model Loaded ------------
	 *Total Params* =  36440528
	 *Trainable Params* =  36440512


The following blocks are all for data pre-processing. Change the random seed to change the train/test sets for validation.

In [14]:
#train_test_ratio = 0.8
#np.random.seed(42)
#indices = np.random.permutation(100000)
#train_index = indices[:int(train_test_ratio*100000)]
#test_index = indices[int(train_test_ratio*100000):]
#train_dataset = Subset(dataset, train_index)
#test_dataset = Subset(dataset, test_index)


In [15]:
#rating, user, item, mask = model.preprocessor(train_dataset, dataset)
#test_rating, _, _, test_mask = model.preprocessor(test_dataset, dataset)

In [16]:
#torch.save(train_mask, '/content/drive/MyDrive/RecSys/1m_mask_nf.pt')
#torch.save(train_rating, '/content/drive/MyDrive/RecSys/1m_rating_nf.pt')
#torch.save(user_matrix, '/content/drive/MyDrive/RecSys/1m_user_nf.pt')
#torch.save(item_matrix, '/content/drive/MyDrive/RecSys/1m_item_nf.pt')
#torch.save(test_rating, '/content/drive/MyDrive/RecSys/1m_test_rating_nf.pt')
#torch.save(test_mask, '/content/drive/MyDrive/RecSys/1m_test_mask_nf.pt')

Loading the saved training data.

In [17]:
train_mask = torch.load('/content/drive/MyDrive/RecSys/1m_mask_nf.pt')
train_rating = torch.load('/content/drive/MyDrive/RecSys/1m_rating_nf.pt')
user_matrix = torch.load('/content/drive/MyDrive/RecSys/1m_user_nf.pt')
item_matrix = torch.load('/content/drive/MyDrive/RecSys/1m_item_nf.pt')
test_rating = torch.load('/content/drive/MyDrive/RecSys/1m_test_rating_nf.pt')
test_mask = torch.load('/content/drive/MyDrive/RecSys/1m_test_mask_nf.pt')

<ipython-input-17-516b6fedb41c>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_mask = torch.load('/content/drive/MyDrive/RecSys/1m_mask_nf.pt')
<ipython-input-17-516



The following train-test split is when we attempt to see how the algorithm deal with the cold start issue. *PLEASE DONT DELETE ANY OF THE CODES*

In [18]:
#train_test_ratio = 0.8
#indices = np.random.permutation(user.shape[0])
#train_index = indices[:int(train_test_ratio*user.shape[0])]
#test_index = indices[int(train_test_ratio*user.shape[0]):]

In [19]:
train_rating = torch.tensor(train_rating)
user_matrix = torch.tensor(user_matrix)
item_matrix = torch.tensor(item_matrix)
item_matrix = torch.nan_to_num(item_matrix, nan=0)
train_mask = torch.tensor(train_mask)
test_rating = torch.tensor(test_rating)
test_mask = torch.tensor(test_mask)
#

In [20]:
print(f'item: {item_matrix.shape}')
print(f'user: {user_matrix.shape}')

item: torch.Size([3952, 19])
user: torch.Size([6040, 39])


**Re-run the following codes for Cross-Validation after running new dataset construction codes at the bottom.**

In [21]:

def custom_collate(batch, batch_size, dataset):
    if len(batch) < batch_size:

        additional_samples_needed = batch_size - len(batch)
        indices = np.random.choice(len(dataset), additional_samples_needed)
        additional_data = [dataset[idx] for idx in indices]
        batch.extend(additional_data)
    return default_collate(batch)


In [22]:
batch_size = model.batch_size

In [23]:
combineset = CombinedDataset(user_matrix, train_rating, train_mask)
combine_testset = CombinedDataset(user_matrix, test_rating, test_mask)
dataset = DataLoader(combineset, batch_size = batch_size, shuffle = True,
                     collate_fn=lambda x: custom_collate(x, batch_size, combineset))
test_dataset = DataLoader(combine_testset, batch_size = batch_size, shuffle = False,
                     collate_fn=lambda x: custom_collate(x, batch_size, combineset))

### Remember to set a unique run_id to make fine-tuning easer

In [24]:
#Reload the drive to update config
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [25]:
model = AEReco(configuration_file = '/content/drive/MyDrive/RecSys/MovieLens1M.yaml')


 	 ----------- Model = Recommender with Autoencoder ------------

 	 	 Autoencoder = DAE_KAN 

 	 ----------- Model Loaded ------------
	 *Total Params* =  36440528
	 *Trainable Params* =  36440512


In [26]:
model.train(dataset, item_matrix, test_dataset, epoch =30)


Epoch 1: 100%|██████████| 12/12 [00:02<00:00,  4.30batch/s, loss=0.335]



	 Epoch : 1, Average Training RMSE = 2.2587
	 Training time for current epoch: 2.8 seconds
	 RMSE on testing set : 1.8667


Epoch 2: 100%|██████████| 12/12 [00:02<00:00,  5.37batch/s, loss=0.196]



	 Epoch : 2, Average Training RMSE = 1.7426
	 Training time for current epoch: 2.24 seconds
	 RMSE on testing set : 1.3817


Epoch 3: 100%|██████████| 12/12 [00:02<00:00,  4.18batch/s, loss=0.131]


...


	 Epoch : 28, Average Training RMSE = 0.9945
	 Training time for current epoch: 2.25 seconds
	 RMSE on testing set : 0.9816


Epoch 29: 100%|██████████| 12/12 [00:02<00:00,  5.41batch/s, loss=0.0824]



	 Epoch : 29, Average Training RMSE = 0.995
	 Training time for current epoch: 2.22 seconds
	 RMSE on testing set : 0.9826


Epoch 30: 100%|██████████| 12/12 [00:02<00:00,  4.49batch/s, loss=0.0874]



	 Epoch : 30, Average Training RMSE = 0.9976
	 Training time for current epoch: 2.68 seconds
	 RMSE on testing set : 1.005
checkpoint saved at epoch 30.
